<a href="https://colab.research.google.com/github/shubham-garg1/mmd/blob/main/MMD_Poject_NASNetMobile_to_be_added_to_main_repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting GCS Bucket:

In [1]:
# !pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 915.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling 

In [2]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

# Install Cloud Storage FUSE.
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0   9240      0 --:--:-- --:--:-- --:--:--  9264
OK
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 5,413 kB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121666 files and directories currently installed.)
Preparing to unpack .../gcsfuse_1.3.0_amd64.deb ...
Unpacking gcsfuse (1.3.0) ...
Setting up 

In [3]:
# Mount a Cloud Storage bucket or location, without the gs:// prefix.
mount_path = "my-mmd"  # or a location like "my-bucket/path/to/mount"
local_path = f"/content/{mount_path}"

!mkdir -p {local_path}
!gcsfuse --implicit-dirs {mount_path} {local_path}

{"time":"14/12/2023 10:56:46.806635","severity":"INFO","msg":"Start gcsfuse/1.3.0 (Go version go1.21.4) for app \"\" using mount point: /content/my-mmd\n"}


You can mount an entire bucket, or a path location within that bucket.
The local path to mount it must exist.

In [4]:
# Then you can access it like a local path.
!ls -lh {local_path}

total 0
drwxr-xr-x 1 root root 0 Dec 14 22:56 batch_dataset
drwxr-xr-x 1 root root 0 Dec 14 22:56 final-yr-proj
drwxr-xr-x 1 root root 0 Dec 14 22:56 results
drwxr-xr-x 1 root root 0 Dec 14 22:56 test_data
drwxr-xr-x 1 root root 0 Dec 14 22:56 train_data
drwxr-xr-x 1 root root 0 Dec 14 22:56 val_dataset


# Loading data:

In [5]:
# # Calculate the split indices
# train_size = 625
# print(train_size)
# # Split the dataset

# SEED = 42 # NOTE: with no seed, you overlap train and test!

# train_data = batch_dataset.shuffle(1000,seed=SEED).take(train_size)
# test_data = batch_dataset.shuffle(1000,seed=SEED).skip(train_size)

In [6]:
import tensorflow as tf
import os
# Load the dataset
train_data = tf.data.experimental.load('gs://my-mmd/train_data')

# Print the dataset
for element,l in train_data:
    print(l.numpy().tolist())
    break

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


[0, 0, 3, 3, 4, 3, 2, 0, 4, 4, 1, 0, 3, 1, 2, 2, 2, 3, 4, 0, 3, 1, 1, 3, 1, 3, 3, 2, 0, 0, 3, 4]


In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [8]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [9]:
import tensorflow as tf
import os
# Define a callback to print metrics
class PrintMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\nEpoch {epoch+1}: {logs}")
        try:
            print(h)
        except:
            pass
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)
# Train the model
epochs = 10

In [10]:
tf.version.VERSION

'2.15.0'

In [11]:
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'

In [12]:
import tensorflow as tf

with strategy.scope():
    test_data = tf.data.Dataset.load("gs://my-mmd/test_data")
    train_data = tf.data.Dataset.load("gs://my-mmd/train_data")
    val_dataset = tf.data.Dataset.load("gs://my-mmd/val_dataset")

In [13]:
input_shape = (256,256,3)
len(train_data)

625

# Defining the CustomNetSmall (1 attention layer) model:

In [14]:
def model_name_formatter(base_model_name):
    result = ''
    capitalize_next = True  # Flag to determine when to capitalize the character
    for char in base_model_name:
        if char.isdigit():
            result += char
            capitalize_next = True
        elif char.isalpha():
            if capitalize_next:
                result += char.upper()
                capitalize_next = False
            else:
                result += char
        else:
            result += char
    return result.replace('net','Net')

In [15]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention, Reshape, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50

# Define the input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 5

# Transformer Encoder Layer
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = tf.add(x, inputs)

    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return tf.add(x, res)

# Building the Model
def build_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # ResNet Base
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    x = base_model.output
    x = Flatten()(x)  # Flatten the output of the convolutions
    x = Reshape((-1, x.shape[-1]))(x)  # Prepare for the Transformer

    # Transformer Encoders
    for _ in range(1):  # You can adjust the number of Transformer layers
        x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=256, dropout=0.1)

    # Global Average Pooling
    x = GlobalAveragePooling1D()(x)

    # Classification Head
    for units in [1024, 512, 256]:  # You can adjust the size and number of Dense layers
        x = Dense(units, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = BatchNormalization()(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    # Create and compile the model
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Add, Flatten, Reshape, GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model

# Define the input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 5

# Transformer Encoder Layer
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = Add()([x, inputs])

    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return Add()([x, res])

# CNN Block
def conv_block(x, filters, kernel_size, strides=(1, 1), activation='relu'):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    return Activation(activation)(x)

# Residual Block
def residual_block(x, filters, kernel_size, strides=(1, 1)):
    shortcut = x
    x = conv_block(x, filters, kernel_size, strides)
    x = conv_block(x, filters, kernel_size, strides)
    shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(shortcut)
    shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    return Activation('relu')(x)

# Custom Model
def build_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Initial Convolutional Layers
    x = conv_block(inputs, 64, (7, 7), strides=(2, 2))
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Multiple Residual Blocks
    for filters in [64, 128]:
        x = residual_block(x, filters, (3, 3))
        x = residual_block(x, filters, (3, 3))

    # Flatten and reshape for transformer
    x = Flatten()(x)
    x = Reshape((-1, 64))(x)  # Reshape to (sequence_length, feature_dim)

    # Multiple Transformer Encoders
    for _ in range(2):  # Increase or decrease the range for depth
        x = transformer_encoder(x, head_size=64, num_heads=8, ff_dim=2048, dropout=0.1)

    # Global Average Pooling for sequence data
    x = GlobalAveragePooling1D()(x)

    # Dense Classifier
    for units in [1024, 512, 256]:
        x = Dense(units, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = BatchNormalization()(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs=x)
    return model




In [17]:
import gc
tf.keras.backend.clear_session()
with strategy.scope():
    gc.collect()
    # MENTION MODEL HERE (If defining custom model using tf.keras.Sequential(...), make sure to change the model.name to an appropriate name using model._name = <NAME> :

with strategy.scope():

# Create and compile the model
    model = build_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Adding model name:
    model._name =  model_name_formatter("LargeCustom")

    # Model Summary
    model.summary()


Model: "LargeCustom"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 64)         0         ['batch_normalizatio

# Training the CustomNetSmall (1 attention layer) Model:

In [18]:
import tensorflow as tf

class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=5):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_combined_metric = float('-inf')  # Initialize with a very small value
        self.wait = 0  # Counter to track the number of epochs without improvement
        self.epochs_before_stopping = 0
        self.best_weights = None  # Variable to store the best weights

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}

        val_accuracy = logs.get('val_accuracy')
        val_loss = logs.get('val_loss')

        self.epochs_before_stopping += 1

        if val_accuracy is None or val_loss is None:
            print("Validation accuracy or validation loss is not found in logs. Cannot calculate custom metric.")
            return

        combined_metric = val_accuracy / (val_loss ** 2)

        if combined_metric > self.best_combined_metric:
            self.best_combined_metric = combined_metric
            self.wait = 0  # Reset the counter when a new best metric is found
            self.best_weights = self.model.get_weights()  # Save the current best weights
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"Stopping training as the combined metric didn't improve for {self.patience} epochs.")
                if self.best_weights is not None:
                    print("Restoring best weights.")
                    self.model.set_weights(self.best_weights)  # Restore the best weights
                self.model.stop_training = True


In [19]:
import os
from tensorflow.python.profiler import profiler_client

tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
print(profiler_client.monitor(tpu_profile_service_address, 100, 2))

  Timestamp: 22:57:26
  TPU type: TPU v2
  Utilization of TPU Matrix Units (higher is better): 0.000%




In [20]:
model.summary()

Model: "LargeCustom"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 64)         0         ['batch_normalizatio

In [22]:

with strategy.scope():
    earlyStopping = CustomEarlyStopping(patience=30)
    # Train the model
    h = model.fit(train_data,
                  epochs=500,
                  validation_data=val_dataset,
                  callbacks=[
                      # tboard_callback,
                      PrintMetricsCallback(),
                      earlyStopping
                  #     EpochModelCheckpoint("./att_model_checkpoints/epoch{epoch:02d}", frequency=1)
                  ]
              )


Epoch 1/500


ResourceExhaustedError: ignored

# Evaluating the CustomNetSmall (1 attention layer) results:

In [ ]:
with strategy.scope():

    train_loss, train_accuracy = model.evaluate(train_data)
    print(f'Train accuracy: {train_accuracy}')

In [ ]:
with strategy.scope():

    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f'Validation accuracy: {val_accuracy}')

In [ ]:
with strategy.scope():

    test_loss, test_accuracy = model.evaluate(test_data)
    print(f'Test accuracy: {test_accuracy}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
with strategy.scope():
    y_true = []
    y_pred = []

    for i,features in test_data:
        images, labels = i,features
        predictions = model.predict(images, verbose=0)
        y_true.extend(labels.numpy())
        y_pred.extend(tf.argmax(predictions, axis=1).numpy())

    # Compute the metrics
    accuracy = sum([1 for true, pred in zip(y_true, y_pred) if true == pred]) / len(y_true)
    loss = model.evaluate(test_data)
    classification_report = classification_report(y_true, y_pred)
    confusion_mat = confusion_matrix(y_true, y_pred)

    print("Accuracy:", accuracy)
    print("Loss:", loss)

In [ ]:
with strategy.scope():
    print("Classification Report:\n", classification_report)

In [ ]:
with strategy.scope():
    print("Confusion Matrix:\n", confusion_mat)

In [ ]:
import seaborn as sns
with strategy.scope():
    sns.heatmap(confusion_mat, square=True, annot=True, cmap='Blues', fmt='d', cbar=False)

In [ ]:
with strategy.scope():

    from sklearn.metrics import classification_report
    import pandas as pd

    clf_report = classification_report(y_true, y_pred,
                                      labels=range(5),
                                      target_names=range(5),
                                      output_dict=True)

    sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)

In [ ]:
with strategy.scope():
    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Plot Loss
    y_loss = h.history["loss"]
    axs[0, 0].plot(y_loss, color='blue', label='Training Loss')
    axs[0, 0].set_title('Loss')
    axs[0, 0].set_xlabel('Epochs')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].grid(True)
    axs[0, 0].legend()

    # Plot Accuracy
    y_accuracy = h.history["accuracy"]
    axs[0, 1].plot(y_accuracy, color='orange', label='Training Accuracy')
    axs[0, 1].set_title('Accuracy')
    axs[0, 1].set_xlabel('Epochs')
    axs[0, 1].set_ylabel('Accuracy')
    axs[0, 1].grid(True)
    axs[0, 1].legend()

    # Plot Validation Accuracy
    y_val_accuracy = h.history["val_accuracy"]
    axs[1, 0].plot(y_val_accuracy, color='green', label='Validation Accuracy')
    axs[1, 0].set_title('Validation Accuracy')
    axs[1, 0].set_xlabel('Epochs')
    axs[1, 0].set_ylabel('Accuracy')
    axs[1, 0].grid(True)
    axs[1, 0].legend()

    # Plot Validation Loss
    y_val_loss = h.history["val_loss"]
    axs[1, 1].plot(y_val_loss, color='red', label='Validation Loss')
    axs[1, 1].set_title('Validation Loss')
    axs[1, 1].set_xlabel('Epochs')
    axs[1, 1].set_ylabel('Loss')
    axs[1, 1].grid(True)
    axs[1, 1].legend()

    plt.tight_layout()
    plt.show()


In [ ]:
with strategy.scope():
    import matplotlib.pyplot as plt

    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Plot Loss
    y_loss = h.history["loss"]
    axs[0, 0].plot(y_loss)
    axs[0, 0].set_title('Loss')

    # Plot Accuracy
    y_accuracy = h.history["accuracy"]
    axs[0, 1].plot(y_accuracy)
    axs[0, 1].set_title('Accuracy')

    # Plot Validation Accuracy
    y_val_accuracy = h.history["val_accuracy"]
    axs[1, 0].plot(y_val_accuracy)
    axs[1, 0].set_title('Validation Accuracy')

    # Plot Validation Loss
    y_val_loss = h.history["val_loss"]
    axs[1, 1].plot(y_val_loss)
    axs[1, 1].set_title('Validation Loss')

    plt.tight_layout()
    plt.show()


In [ ]:
with strategy.scope():

    import matplotlib.pyplot as plt

    y = h.history["loss"]
    plt.plot(y)
    plt.title("Loss")
    plt.show()

In [ ]:
with strategy.scope():

    import matplotlib.pyplot as plt

    y = h.history["accuracy"]
    plt.plot(y)
    plt.title("Accuracy")
    plt.show()

In [ ]:
with strategy.scope():

    import matplotlib.pyplot as plt

    y = h.history["val_accuracy"]
    plt.plot(y)
    plt.title("Validation Accuracy")
    plt.show()

In [ ]:
with strategy.scope():

    import matplotlib.pyplot as plt

    y = h.history["val_loss"]
    plt.plot(y)
    plt.title("Validation Loss")
    plt.show()

# Sending CustomNetSmall (1 attention layer) results to GCS:

In [ ]:
df = pd.DataFrame(h.history)
df

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
try:
    f1 = clf_report['weighted avg']['f1-score']
    x = "trainable" if base_model.trainable else "not_trainable"
    df.to_csv(f"/content/my-mmd/results/tpu/{model.name}_base_model_{x}_{f1}_f1_results.csv", index=False)
    model.save(f'/content/my-mmd/results/tpu/models/{model.name}_base_model_{x}_{f1}_f1.h5', options=save_locally)
except:
    df.to_csv(f"/content/my-mmd/results/tpu/{model.name}_results.csv", index=False)
    model.save(f'/content/my-mmd/results/tpu/models/{model.name}.h5', options=save_locally)

# Sending CustomNetSmall (1 attention layer) data to spreadsheet (VERY IMPORTANT):

In [ ]:
# Sendind data to spreadseet:
with strategy.scope():

    spreadsheet_id = "1ceVqsaNC5GZ2bcduY8ULZW9NlZlcYDfS6wWhEKLOZd4"
    import gspread
    from google.auth import default
    creds, _ = default()

    c = gspread.authorize(creds)

    wb = c.open_by_key(spreadsheet_id)

    # Accessing precision, recall, and F1-score for all classes combined (last row)
    precision = clf_report['weighted avg']['precision']
    recall = clf_report['weighted avg']['recall']
    f1_score = clf_report['weighted avg']['f1-score']

    # Printing the aggregated metrics for all classes combined
    print(f"Weighted Average - Precision: {precision}, Recall: {recall}, F1-score: {f1_score}")

    # Constructing the row data to be appended
    try:
        row_data = [
            '21',  # String value
            base_model.name,  # String value
            'Yes' if base_model.trainable else 'No',
            '500',
            str(earlyStopping.epochs_before_stopping),
            str(round(float(train_accuracy),4)),  # Convert to string if not already
            str(round(float(train_loss),4)),  # Convert to string if not already
            str(round(float(val_accuracy),4)),  # Convert to string if not already
            str(round(float(val_loss),4)),  # Convert to string if not already
            str(round(float(test_accuracy),4)),  # Convert to string if not already
            str(round(float(test_loss),4)),  # Convert to string if not already
            str(round(float(precision),4)),
            str(round(float(recall),4)),
            str(round(float(f1_score),4))
        ]
    except:
        row_data = [
            '21',  # String value
            model.name,  # String value
            'Yes' if model.trainable else 'No',
            '500',
            str(earlyStopping.epochs_before_stopping),
            str(round(float(train_accuracy),4)),  # Convert to string if not already
            str(round(float(train_loss),4)),  # Convert to string if not already
            str(round(float(val_accuracy),4)),  # Convert to string if not already
            str(round(float(val_loss),4)),  # Convert to string if not already
            str(round(float(test_accuracy),4)),  # Convert to string if not already
            str(round(float(test_loss),4)),  # Convert to string if not already
            str(round(float(precision),4)),
            str(round(float(recall),4)),
            str(round(float(f1_score),4))
        ]

    ws = wb.worksheet('Sheet1')

    ws.append_row(row_data)

In [ ]:
# If above code doesn't automatically update the values, copy the values from here and Ctrl+Shift+V on Spreadsheet to paste the values:

import math
for x in row_data:
  try:
    print(round(float(x),4), end="\t")
  except:
    print(x, end = "\t")